In [62]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import Data
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import GCNConv
from torch_geometric.utils.convert import from_networkx, to_networkx

In [63]:
df = pd.read_csv("./data/Nabil Bank Limited ( NABIL ).csv")
df = df.drop(columns=["published_date", "status"])
df["per_change"] = df["per_change"].fillna(0)

In [64]:
# print(df.info())
# print(df.describe())
# col_num = len(df.columns)
# fig, axes = plt.subplots(col_num, 1, figsize=(6, 4 * col_num), constrained_layout=True)

# for i, column in enumerate(df.columns):
#     sns.histplot(df[column], kde=True, ax=axes[i])
#     axes[i].set_title(f"Histogram of {column} column")
#     axes[i].set_xlabel(column)
#     axes[i].set_ylabel("Frequency")
# plt.show()

In [65]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_data)

data = scaled_df[:-1]
target = scaled_df.shift(-1)[:-1]

node_features = torch.tensor(data.values, dtype=torch.float)
target_tensor = torch.tensor(target.values, dtype=torch.float)

num_nodes = len(node_features)

edge_index = torch.tensor(
    [[i, i + 1] for i in range(num_nodes - 1)]
    + [[i + 1, i] for i in range(num_nodes - 1)],
    dtype=torch.long,
).t()

td_amt = data[6].to_list()

g = nx.visibility_graph(td_amt)
temp = from_networkx(g)
graph = Data(x=node_features, edge_index=temp.edge_index)

In [66]:
class GNNForecastor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GNNForecastor, self).__init__()
        self.conv1 = GCNConv(input_size, hidden_size)
        self.conv2 = GCNConv(hidden_size, hidden_size)
        self.lin = nn.Linear(hidden_size, output_size)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.lin(x)
        return x


input_size = node_features.shape[1]
hidden_size = 16
output_size = input_size
epochs = 100
learning_rate = 1e-2

model = GNNForecastor(input_size, hidden_size, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
ls_fn = nn.MSELoss()

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    output = model(graph.x, graph.edge_index)

    loss = ls_fn(output, target_tensor)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch +1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/100, Loss: 1.0530
Epoch 2/100, Loss: 0.9954
Epoch 3/100, Loss: 0.9499
Epoch 4/100, Loss: 0.9097
Epoch 5/100, Loss: 0.8675
Epoch 6/100, Loss: 0.8217
Epoch 7/100, Loss: 0.7722
Epoch 8/100, Loss: 0.7200
Epoch 9/100, Loss: 0.6658
Epoch 10/100, Loss: 0.6107
Epoch 11/100, Loss: 0.5565
Epoch 12/100, Loss: 0.5056
Epoch 13/100, Loss: 0.4597
Epoch 14/100, Loss: 0.4212
Epoch 15/100, Loss: 0.3924
Epoch 16/100, Loss: 0.3736
Epoch 17/100, Loss: 0.3615
Epoch 18/100, Loss: 0.3517
Epoch 19/100, Loss: 0.3423
Epoch 20/100, Loss: 0.3343
Epoch 21/100, Loss: 0.3299
Epoch 22/100, Loss: 0.3293
Epoch 23/100, Loss: 0.3303
Epoch 24/100, Loss: 0.3294
Epoch 25/100, Loss: 0.3250
Epoch 26/100, Loss: 0.3181
Epoch 27/100, Loss: 0.3108
Epoch 28/100, Loss: 0.3048
Epoch 29/100, Loss: 0.3008
Epoch 30/100, Loss: 0.2985
Epoch 31/100, Loss: 0.2972
Epoch 32/100, Loss: 0.2964
Epoch 33/100, Loss: 0.2956
Epoch 34/100, Loss: 0.2948
Epoch 35/100, Loss: 0.2941
Epoch 36/100, Loss: 0.2931
Epoch 37/100, Loss: 0.2919
Epoch 38/1

In [67]:
model.eval()
with torch.no_grad():
    predicted = model(graph.x, graph.edge_index)

predicted_og = scaler.inverse_transform(predicted.numpy())
target_og = scaler.inverse_transform(target_tensor.numpy())

predicted_og_round = np.round(predicted_og, 2)
target_og_round = np.round(target_og, 2)

np.set_printoptions(precision=2, suppress=True, linewidth=100)
for i in range(num_nodes):
    print(f"Predicted: {predicted_og_round[i]}")
    print(f"   Actual: {target_og_round[i]}")
    print("")

Predicted: [   1194.77    1255.12    1187.86    1252.37       1.34   -2433.57 2022720.5 ]
   Actual: [  1155.     1193.     1160.     1193.        3.29    156.   183026.55]

Predicted: [   1194.77    1255.12    1187.86    1252.37       1.34   -2433.57 2022720.5 ]
   Actual: [   1193.      1172.      1150.      1172.        -1.76     939.   1087972.5 ]

Predicted: [   1014.8     1042.38    1019.16    1010.18      -0.12    7856.43 8535047.  ]
   Actual: [  1172.     1194.     1155.     1194.        1.88    490.   573992.56]

Predicted: [    1062.04     1042.69     1054.15     1038.26       -0.49    12830.98 13871767.  ]
   Actual: [  1194.     1193.     1178.     1178.       -1.34    115.   135930.55]

Predicted: [    1056.85     1055.43     1041.47     1046.16        0.04    12286.1  13335439.  ]
   Actual: [  1178.     1189.     1165.     1170.       -0.68    239.   279926.56]

Predicted: [    1065.9      1056.76     1051.48     1050.87       -0.15    12397.87 13705225.  ]
   Actual: [